# 01 - Advanced Feature Engineering

This notebook implements sophisticated feature engineering for AI agent performance analysis.

**Course:** DATA 230 (Data Visualization) at SJSU

## Features Created:
1. **Business-Centric Features**: business_value_score, operational_risk_index, scalability_potential, total_cost_of_ownership
2. **Temporal Intelligence**: performance_trend_7d, stability_index, degradation_risk_score, seasonality_impact
3. **Strategic Groupings**: performance_quartile, cost_efficiency_tier, strategic_importance


In [1]:
import pandas as pd
import numpy as np

# Load cleaned data
df = pd.read_csv('../data/cleaned/cleaned_data.csv')
print(f"Loaded {len(df)} records")
print(f"Columns: {list(df.columns)}")


Loaded 5000 records
Columns: ['agent_id', 'agent_type', 'model_architecture', 'deployment_environment', 'task_category', 'task_complexity', 'autonomy_level', 'success_rate', 'accuracy_score', 'efficiency_score', 'execution_time_seconds', 'response_latency_ms', 'memory_usage_mb', 'cpu_usage_percent', 'cost_per_task_cents', 'human_intervention_required', 'error_recovery_rate', 'multimodal_capability', 'edge_compatibility', 'privacy_compliance_score', 'bias_detection_score', 'timestamp', 'data_quality_score', 'performance_index', 'cost_efficiency_ratio', 'autonomous_capability_score']


## 1. Business-Centric Features


In [2]:
# Convert human_intervention_required to numeric (frequency proxy)
df['human_intervention_frequency'] = df['human_intervention_required'].astype(int)

# Calculate cost_efficiency and resource_efficiency for use in business_value_score
df['cost_efficiency'] = df['cost_efficiency_ratio'] / df['cost_efficiency_ratio'].max()
df['resource_efficiency'] = df['efficiency_score'] / (df['memory_usage_mb'] * df['cpu_usage_percent']) * 1000
df['resource_efficiency'] = df['resource_efficiency'] / df['resource_efficiency'].max()

# business_value_score: Weighted combination
df['business_value_score'] = (
    df['success_rate'] * 0.25 +
    df['cost_efficiency'] * 0.20 +
    df['resource_efficiency'] * 0.15 +
    (1 - df['human_intervention_frequency']) * 0.20 +
    df['error_recovery_rate'] * 0.10 +
    df['privacy_compliance_score'] * 0.10
)

print("business_value_score created")
print(df['business_value_score'].describe())


business_value_score created
count    5000.000000
mean        0.334121
std         0.137408
min         0.190440
25%         0.234735
50%         0.295188
75%         0.365727
max         0.862132
Name: business_value_score, dtype: float64


In [3]:
# operational_risk_index: Combination of failure_probability + maintenance_complexity + dependency_score
df['failure_probability'] = 1 - df['success_rate']
df['maintenance_complexity'] = df['task_complexity'] / 10 * (1 - df['error_recovery_rate'])
df['dependency_score'] = (1 - df['autonomous_capability_score'] / 100) * df['human_intervention_frequency']

df['operational_risk_index'] = (
    df['failure_probability'] +
    df['maintenance_complexity'] +
    df['dependency_score']
)

print("operational_risk_index created")
print(df['operational_risk_index'].describe())


operational_risk_index created
count    5000.000000
mean        0.792723
std         0.313032
min         0.033500
25%         0.566540
50%         0.821600
75%         1.044957
max         1.472300
Name: operational_risk_index, dtype: float64


In [4]:
# scalability_potential: Based on performance trends and resource efficiency
df['scalability_potential'] = (
    df['performance_index'] * df['resource_efficiency'] * 
    (1 - df['cpu_usage_percent'] / 100) * 
    (1 - df['memory_usage_mb'] / df['memory_usage_mb'].max())
)

print("scalability_potential created")
print(df['scalability_potential'].describe())


scalability_potential created
count    5000.000000
mean        0.019100
std         0.039798
min        -0.000810
25%         0.000811
50%         0.004653
75%         0.018307
max         0.526337
Name: scalability_potential, dtype: float64


In [5]:
# total_cost_of_ownership: (cost_per_task * execution_frequency) + (maintenance_cost * complexity)
# Proxy execution_frequency from execution_time (higher time = lower frequency possible)
df['execution_frequency_proxy'] = 1 / df['execution_time_seconds']
df['maintenance_cost_proxy'] = df['human_intervention_frequency'] * df['response_latency_ms'] / 1000

df['total_cost_of_ownership'] = (
    df['cost_per_task_cents'] * df['execution_frequency_proxy'] +
    df['maintenance_cost_proxy'] * df['task_complexity']
)

print("total_cost_of_ownership created")
print(df['total_cost_of_ownership'].describe())


total_cost_of_ownership created
count    5000.000000
mean        5.196098
std         5.793033
min         0.000366
25%         1.352660
50%         3.369967
75%         6.964168
max        48.126353
Name: total_cost_of_ownership, dtype: float64


## 2. Temporal Intelligence


In [6]:
# Parse timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour_of_day'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek

# performance_trend_7d: Slope of performance metrics over time (rolling window proxy)
# Group by agent_type and calculate rolling mean difference as trend proxy
df = df.sort_values(['agent_type', 'timestamp'])
df['performance_trend_7d'] = df.groupby('agent_type')['performance_index'].transform(
    lambda x: x.diff().rolling(window=7, min_periods=1).mean()
)
df['performance_trend_7d'] = df['performance_trend_7d'].fillna(0)

print("performance_trend_7d created")
print(df['performance_trend_7d'].describe())


performance_trend_7d created
count    5000.000000
mean        0.000010
std         0.028892
min        -0.227180
25%        -0.018932
50%         0.000124
75%         0.018510
max         0.300470
Name: performance_trend_7d, dtype: float64


In [7]:
# stability_index: 1 - (coefficient of variation of success_rate)
# Calculate CV per agent_type
cv_by_agent = df.groupby('agent_type')['success_rate'].transform(
    lambda x: x.std() / x.mean() if x.mean() != 0 else 0
)
df['stability_index'] = 1 - cv_by_agent
df['stability_index'] = df['stability_index'].clip(lower=0, upper=1)

print("stability_index created")
print(df['stability_index'].describe())


stability_index created
count    5000.000000
mean        0.676363
std         0.009080
min         0.658132
25%         0.670768
50%         0.677628
75%         0.682751
max         0.690483
Name: stability_index, dtype: float64


In [8]:
# degradation_risk_score: Probability of performance decline based on historical patterns
# Higher risk if: low stability, negative trend, high failure probability
df['degradation_risk_score'] = (
    (1 - df['stability_index']) * 0.4 +
    (-df['performance_trend_7d'].clip(upper=0)) * 0.3 +
    df['failure_probability'] * 0.3
)
df['degradation_risk_score'] = df['degradation_risk_score'].clip(lower=0, upper=1)

print("degradation_risk_score created")
print(df['degradation_risk_score'].describe())


degradation_risk_score created
count    5000.000000
mean        0.285585
std         0.050500
min         0.134288
25%         0.245746
50%         0.290290
75%         0.333733
max         0.399184
Name: degradation_risk_score, dtype: float64


In [9]:
# seasonality_impact: Performance variation by hour/day patterns
# Calculate mean performance by hour and day, then deviation from overall mean
hourly_perf = df.groupby('hour_of_day')['performance_index'].transform('mean')
daily_perf = df.groupby('day_of_week')['performance_index'].transform('mean')
overall_mean = df['performance_index'].mean()

df['seasonality_impact'] = (
    (hourly_perf - overall_mean).abs() / overall_mean +
    (daily_perf - overall_mean).abs() / overall_mean
) / 2

print("seasonality_impact created")
print(df['seasonality_impact'].describe())


seasonality_impact created
count    5000.000000
mean        0.005090
std         0.003032
min         0.001982
25%         0.002909
50%         0.004306
75%         0.006259
max         0.011875
Name: seasonality_impact, dtype: float64


## 3. Strategic Groupings


In [10]:
# performance_quartile: Relative performance within agent_type
df['performance_quartile'] = df.groupby('agent_type')['performance_index'].transform(
    lambda x: pd.qcut(x, q=4, labels=[1, 2, 3, 4], duplicates='drop')
)
df['performance_quartile'] = df['performance_quartile'].astype(int)

print("performance_quartile created")
print(df['performance_quartile'].value_counts().sort_index())


performance_quartile created
performance_quartile
1    1255
2    1248
3    1246
4    1251
Name: count, dtype: int64


In [11]:
# cost_efficiency_tier: Cost-performance ranking
df['cost_efficiency_tier'] = pd.qcut(
    df['cost_efficiency_ratio'], 
    q=5, 
    labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'],
    duplicates='drop'
)

print("cost_efficiency_tier created")
print(df['cost_efficiency_tier'].value_counts())


cost_efficiency_tier created
cost_efficiency_tier
Very Low     1000
Low          1000
Medium       1000
High         1000
Very High    1000
Name: count, dtype: int64


In [12]:
# strategic_importance: Business impact classification
# Based on business_value_score and task_complexity
def classify_importance(row):
    if row['business_value_score'] >= 0.7 and row['task_complexity'] >= 7:
        return 'Critical'
    elif row['business_value_score'] >= 0.5 and row['task_complexity'] >= 5:
        return 'High'
    elif row['business_value_score'] >= 0.3:
        return 'Medium'
    else:
        return 'Low'

df['strategic_importance'] = df.apply(classify_importance, axis=1)

print("strategic_importance created")
print(df['strategic_importance'].value_counts())


strategic_importance created
strategic_importance
Low       2584
Medium    2409
High         7
Name: count, dtype: int64


## Summary and Save


In [13]:
# List all new features created
new_features = [
    # Business-Centric
    'business_value_score',
    'operational_risk_index',
    'scalability_potential',
    'total_cost_of_ownership',
    # Temporal Intelligence
    'performance_trend_7d',
    'stability_index',
    'degradation_risk_score',
    'seasonality_impact',
    # Strategic Groupings
    'performance_quartile',
    'cost_efficiency_tier',
    'strategic_importance'
]

print(f"Total new features created: {len(new_features)}")
print(f"\nNew features: {new_features}")
print(f"\nFinal dataset shape: {df.shape}")


Total new features created: 11

New features: ['business_value_score', 'operational_risk_index', 'scalability_potential', 'total_cost_of_ownership', 'performance_trend_7d', 'stability_index', 'degradation_risk_score', 'seasonality_impact', 'performance_quartile', 'cost_efficiency_tier', 'strategic_importance']

Final dataset shape: (5000, 47)


In [14]:
# Drop intermediate calculation columns
columns_to_drop = [
    'human_intervention_frequency',
    'cost_efficiency',
    'resource_efficiency',
    'failure_probability',
    'maintenance_complexity',
    'dependency_score',
    'execution_frequency_proxy',
    'maintenance_cost_proxy',
    'hour_of_day',
    'day_of_week'
]
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Handle any infinities
df.replace([np.inf, -np.inf], np.nan, inplace=True)

print(f"Final dataset shape after cleanup: {df.shape}")


Final dataset shape after cleanup: (5000, 37)


In [15]:
# Save enhanced dataset
df.to_csv('../data/ml/strategic_agent_features.csv', index=False)
print("Saved to data/ml/strategic_agent_features.csv")


Saved to data/ml/strategic_agent_features.csv
